In [1]:
import os
import cv2
import numpy as np

In [2]:
default_dir = "C:/anoosheh/CP/Master/Thesis/Dataset/Dataset_under_Delta_robot_without_light_0_degree/Prepare_for_yolov8_with_augmentation_with_hand"
input_dir = default_dir + '/masks'
output_dir = default_dir + '/labels'

In [4]:
def get_limits(color):
    c = np.uint8([[color]]) # here insert the bgr values which you want to convert to hsv
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    lowerLimit = hsvC[0][0][0], 40, 0
    upperLimit = hsvC[0][0][0], 255, 255

    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [ ]:
def for_gray_hand():
    lowerLimit = 0, 0, 5
    upperLimit = 0, 0, 255

    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [5]:
def find_mask(mask, cls_num, output_dir, j):
    H, W = mask.shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygon = []
    # convert the contours to polygons
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'a') as f:
        for p_, p in enumerate(polygon):
            if p_ == (len(polygon) - 1):
                f.write('{}\n'.format(p))
            elif p_ == 0:
                f.write(str(cls_num) + ' {} '.format(p))
            else:
                f.write('{} '.format(p))
        f.close()

In [6]:
# in BGR colorspace
yellow = [0, 255, 255]
red = [0, 0, 255]
green = [0, 255, 0]
blue = [255, 0, 0]
gray = [216, 216, 216]

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path)
    hsvMask = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)

    lowerLimit_yellow, upperLimit_yellow = get_limits(yellow)
    lowerLimit_red, upperLimit_red = get_limits(red)
    lowerLimit_green, upperLimit_green = get_limits(green)
    lowerLimit_blue, upperLimit_blue = get_limits(blue)
    lowerLimit_gray, upperLimit_gray = for_gray_hand()

    mask_yellow = cv2.inRange(hsvMask, lowerLimit_yellow, upperLimit_yellow)
    mask_red = cv2.inRange(hsvMask, lowerLimit_red, upperLimit_red)
    mask_green = cv2.inRange(hsvMask, lowerLimit_green, upperLimit_green)
    mask_blue = cv2.inRange(hsvMask, lowerLimit_blue, upperLimit_blue)
    mask_gray = cv2.inRange(hsvMask, lowerLimit_gray, upperLimit_gray)

    if np.sum(mask_red == 255) != 0:
        find_mask(mask_red, 0, output_dir, j)

    if np.sum(mask_blue == 255) != 0:
        find_mask(mask_blue, 1, output_dir, j)

    if np.sum(mask_green == 255) != 0:
        find_mask(mask_green, 2, output_dir, j)

    if np.sum(mask_yellow == 255) != 0:
        find_mask(mask_yellow, 3, output_dir, j)

    if np.sum(mask_gray == 255) != 0:
        find_mask(mask_gray, 4, output_dir, j)

cv2.destroyAllWindows()